In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
from dataclasses import dataclass
import random
from ortools.sat.python import cp_model

# Getting People

In [2]:
def get_form_responses():
    url = r"https://docs.google.com/spreadsheets/d/1KmJBO5oKwygn-2AzwX-hS1wGeQMmLwyizFqtsSZo7_w/export?format=csv"
    df = pd.read_csv(url)
    return df

resps = get_form_responses()
resps

,Timestamp,UCLA email,First and Last name,What year are you?,Have you played with UCLA spike before?,Phone number,What is your skill level,Are you interested in playing competitively or just for fun?,Do you follow ucla.spike on instagram,How did you hear about us?,Feel free to share anything relevant,What days of Tryouts do you plan on coming? (check all that apply),Campaign status,Campaign status.1
0,9/21/2025 9:46:32,youngjakecubes@g.ucla.edu,Jake Young,Senior,Yes,5105420416,6 - Expert: I am willing to bet $100 nobody he...,Sign me up for sectionals,Yes,RecFest,NaN,NaN,EMAIL_SENT,EMAIL_SENT
1,9/22/2025 11:31:56,thiagogarate25@g.ucla.edu,Thiago Garate,Freshman,No,6504008220,2 - Beginner: I have played a bit with friends,Just trying to play for fun,Doing it now,EAF (Enormous Activitys Fair),awesome!,"Tuesday Sept 30, Wed Oct 1",EMAIL_SENT,EMAIL_SENT
2,9/22/2025 11:44:32,ryguy101@g.ucla.edu,Ryan Sung,Junior,No,6268627672,3 - Intermediate: I play with my friends and I...,Just trying to play for fun,Doing it now,EAF (Enormous Activitys Fair),NaN,"Tuesday Sept 30, Wed Oct 1",EMAIL_SENT,EMAIL_SENT
3,9/22/2025 11:45:45,joebruin2324@ucla.edu,joseph claypool,Freshman,No,NaN,3 - Intermediate: I play with my friends and I...,Just trying to play for fun,Doing it now,EAF (Enormous Activitys Fair),NaN,Tuesday Sept 30,EMAIL_SENT,EMAIL_SENT
4,9/22/2025 11:46:55,asaab12@g.ucla.edu,Adam Saab,Freshman,No,NaN,4 - Advanced: I know about competitive serving...,Sign me up for sectionals,Doing it now,EAF (Enormous Activitys Fair),NaN,"Tuesday Sept 30, Wed Oct 1",EMAIL_SENT,EMAIL_SENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,9/29/2025 15:15:51,Justindelama@g.ucla.edu,Justin de Lama,Freshman,No,9094016386,3 - Intermediate: I play with my friends and I...,Sign me up for sectionals,Doing it now,From my amigos,NaN,"Tuesday Sept 30, Wed Oct 1",NaN,NaN
101,9/30/2025 0:11:28,reeceyoshi@g.ucla.edu,Reece Yoshizawa,Senior,Yes,5622805005,6 - Expert: I am willing to bet $100 nobody he...,Sign me up for sectionals,Yes,Miranda,NaN,"Tuesday Sept 30, Wed Oct 1",NaN,NaN
102,9/30/2025 0:14:32,ebstewart13@ucla.edu,Emmett Stewart,Sophomore,Yes,6199224484,5 - Contender: I have played a tournament (not...,Sign me up for sectionals,Yes,I have played with the club before,love you pookies 🥰,"Tuesday Sept 30, Wed Oct 1",NaN,NaN
103,9/30/2025 9:50:41,alexstav13@g.ucla.edu,Alex Stavnitser,Junior,Yes,6504950355,5 - Contender: I have played a tournament (not...,Sign me up for sectionals,Yes,A friend,NaN,Wed Oct 1,NaN,NaN


In [3]:
resps["UCLA email"].value_counts()

UCLA email
youngjakecubes@g.ucla.edu    1
alexiasaigh@g.ucla.edu       1
harlan@ucla.edu              1
camryn@ucla.edu              1
masonchoey@g.ucla.edu        1
                            ..
kevinma13425@g.ucla.edu      1
nikhilv2029@g.ucla.edu       1
svu29@g.ucla.edu             1
amirsky4517@ucla.edu         1
Nyzhou@ucla.edu              1
Name: count, Length: 105, dtype: int64

In [4]:
@dataclass
class Player:
    id: int
    name: str
    email: str
    initial_elo: float
    latest_elo: float = None
    games_played: int = 0

@dataclass
class Game:
    # Player IDs
    round : int
    p1: int
    p2: int
    p3: int
    p4: int
    result: int  # 1 if p1/p2 win, 0 if p3/p4 win

    def get_pids(self):
        return [self.p1, self.p2, self.p3, self.p4]

# Optimizing Ratings

In [5]:
def optimize_ratings(players, games, default_mean = 1000):
    id_to_idx = {player.id: i for i, player in enumerate(players)}
    
    beta = np.log(10) / 400
    n = len(players)
    initial_ratings = np.array([player.initial_elo for player in players]) # modify this according to input skill levels
    sigma = 400.0
    ratings = cp.Variable(n)
    # prior is rating is normal dist with stdev 400, mean self reported
    self_report_prior = -0.5 * cp.sum_squares((ratings - initial_ratings) / sigma)
    mean_prior = 10 * -0.5 * cp.sum_squares((cp.sum(ratings) / n - default_mean) / (sigma / np.sqrt(n)))

    obj_func = []
    for g in games:
        result = float(g.result) # 1 if p1/p2 win, 0 if p3/p4 win
        z = beta * 0.5 * (ratings[id_to_idx[g.p1]] + ratings[[id_to_idx[g.p2]]] - ratings[[id_to_idx[g.p3]]] - ratings[[id_to_idx[g.p4]]])
        obj_func.append(result * z - cp.logistic(z))

    objective = cp.Maximize(cp.sum(obj_func) + self_report_prior) #+ mean_prior)
    constraints = [ratings >= 0, ratings <= 2000]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    #print(ratings.value)
    if True:
        for i in range(n):
            players[i].latest_elo = round(ratings.value[i])
    return ratings.value

# Testing Optimizer

In [6]:
# Josh: 1900 Kyan: 1750 Jake: 1650 Jai: 1625 Archie: 1600 Steven: 1550 Reece: 1450 Luke: 1350 Aubrey: 1250 Alex: 1000
true_elos = {
    "Josh": 1900,
    "Kyan": 1750,
    "Jake": 1650,
    "Jai": 1625,
    "Archie": 1600,
    "Steven": 1550,
    "Aubrey": 1500,
    "Reece": 1450,
    "Luke": 1350,
    "Alex": 1100
}

true_ratings = list(true_elos.values())
names = list(true_elos.keys())

players = [Player(i+10, list(true_elos.keys())[i], "", 1500) for i in range(10)]


In [7]:
print(players)

[Player(id=10, name='Josh', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=11, name='Kyan', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=12, name='Jake', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=13, name='Jai', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=14, name='Archie', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=15, name='Steven', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=16, name='Aubrey', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=17, name='Reece', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=18, name='Luke', email='', initial_elo=1500, latest_elo=None, games_played=0), Player(id=19, name='Alex', email='', initial_elo=1500, latest_elo=None, games_played=0)]


In [8]:
rng = random.Random(101)
num_games = 1000
games = []

for _ in range(num_games):
    # pick 4 distinct players
    picks = rng.sample(players, 4)
    rng.shuffle(picks)
    p1, p2, p3, p4 = picks[0], picks[1], picks[2], picks[3]

    # true win prob for Team1 using Elo model
    D_true = 0.5 * (true_elos[p1.name] + true_elos[p2.name] - true_elos[p3.name] - true_elos[p4.name])
    p_win = 1.0 / (1.0 + 10 ** (-D_true / 400.0))

    # sample outcome
    result = 1 if rng.random() < p_win else 0

    games.append(Game(round = -1, p1=p1.id, p2=p2.id, p3=p3.id, p4=p4.id, result=result))

# ----- Fit ratings with your optimizer -----
estimated = optimize_ratings(players, games, 1500)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cvxpy/problems/problem.py:167: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


In [9]:
print(players)

[Player(id=10, name='Josh', email='', initial_elo=1500, latest_elo=1775, games_played=0), Player(id=11, name='Kyan', email='', initial_elo=1500, latest_elo=1718, games_played=0), Player(id=12, name='Jake', email='', initial_elo=1500, latest_elo=1641, games_played=0), Player(id=13, name='Jai', email='', initial_elo=1500, latest_elo=1559, games_played=0), Player(id=14, name='Archie', email='', initial_elo=1500, latest_elo=1535, games_played=0), Player(id=15, name='Steven', email='', initial_elo=1500, latest_elo=1565, games_played=0), Player(id=16, name='Aubrey', email='', initial_elo=1500, latest_elo=1439, games_played=0), Player(id=17, name='Reece', email='', initial_elo=1500, latest_elo=1438, games_played=0), Player(id=18, name='Luke', email='', initial_elo=1500, latest_elo=1244, games_played=0), Player(id=19, name='Alex', email='', initial_elo=1500, latest_elo=1085, games_played=0)]


# Getting Games

In [10]:
def remove_duplicates(df):
    failure = False
    key_cols = ["Round", "Net_number"]
    agree_cols = ["id1", "id2", "id3", "id4", "Match1Result", "Match2Result", "Match3Result"]
    df = df.copy()

    deduped = []
    for _, group in df.groupby(by = key_cols):
        if len(group) == 1:
            deduped.append(group.iloc[0])
        else:
            first = group.iloc[0]
            for col in agree_cols:
                for _, row in group.iterrows():
                    if not first[col] == row[col]:
                        print("Conflict appeared in the following rows:")
                        print(group.iloc[0])
                        print(row)
                        failure = True
            deduped.append(first)
    if failure:
        return []
    return pd.DataFrame(deduped)


def get_games():
    url = r"https://docs.google.com/spreadsheets/d/1vuQ394gU_CSNEO-U0ZFVRTNC1jIWV1BT_yjGqoQQPOk/export?format=csv"
    df = pd.read_csv(url)

    return df


In [11]:
def df_to_games(df):
    games_total = []
    for _, row in df.iterrows():
        games_total.append(Game(
            round = row["Round"],
            p1 = row["id1"],
            p2 = row["id2"],
            p3 = row["id3"],
            p4 = row["id4"],
            result = row["Match1Result"]
        ))
        games_total.append(Game(
            round = row["Round"],
            p1 = row["id1"],
            p2 = row["id3"],
            p3 = row["id2"],
            p4 = row["id4"],
            result = row["Match2Result"]
        ))
        games_total.append(Game(
            round = row["Round"],
            p1 = row["id1"],
            p2 = row["id4"],
            p3 = row["id2"],
            p4 = row["id3"],
            result = row["Match3Result"]
        ))
    return games_total


In [12]:
def get_game_history():
    df = get_games()
    df = remove_duplicates(df)
    if df.empty:
        print("Conflicts in Game Data or no reported Games")
        return []
    games = df_to_games(df)
    return games

In [13]:
game_hist = get_game_history()
game_hist

[Game(round=1.0, p1=94.0, p2=95.0, p3=97.0, p4=100.0, result=1.0),
 Game(round=1.0, p1=94.0, p2=97.0, p3=95.0, p4=100.0, result=0.0),
 Game(round=1.0, p1=94.0, p2=100.0, p3=95.0, p4=97.0, result=0.0),
 Game(round=1.0, p1=96.0, p2=98.0, p3=99.0, p4=102.0, result=1.0),
 Game(round=1.0, p1=96.0, p2=99.0, p3=98.0, p4=102.0, result=1.0),
 Game(round=1.0, p1=96.0, p2=102.0, p3=98.0, p4=99.0, result=1.0),
 Game(round=1.0, p1=101.0, p2=103.0, p3=104.0, p4=105.0, result=1.0),
 Game(round=1.0, p1=101.0, p2=104.0, p3=103.0, p4=105.0, result=0.0),
 Game(round=1.0, p1=101.0, p2=105.0, p3=103.0, p4=104.0, result=0.0)]

# Posting Pools

In [14]:
def post_pools(game, net_num):
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfaqSgDwyCfvwAM1gSKc7IbGbr14ePMwEJmcBx_fcJB9YVDAg/formResponse"
    import requests
    import time
    try:
        new_row_data = {
            'entry.1134473475': game.round,
            'entry.1834803213': net_num,
            'entry.1013925837': game.p1,
            'entry.2138865174': game.p2,
            'entry.463748120': game.p3,
            'entry.1477352260': game.p4,
        }

        r = requests.post(form_url, data=new_row_data, timeout=10)
        print("Submitted:", game)
        if r.status_code != 200 and r.status_code != 302:
            print("Failed:", r.status_code)
        time.sleep(0.15) 
        return True
        
    except Exception as e:
        print(f"Error submitting results: {e}")
        return False

# Matchmaking

In [15]:
def generate_matchings(players, game_history, round_num):
    print(players)
    id_to_idx = {player.id: i for i, player in enumerate(players)}
    n = len(players)
    assert n % 4 == 0
    num_groups = n // 4

    model = cp_model.CpModel()
    assignments = [model.NewIntVar(0, num_groups - 1, f"assign_{i}") for i in range(n)]
    x = {(i,g): model.NewBoolVar(f"x_{i}_{g}") for i in range(n) for g in range(num_groups)}
    # Channeling: x[i,g] <-> (assignments[i] == g)
    for i in range(n):
        for g in range(num_groups):
            model.Add(assignments[i] == g).OnlyEnforceIf(x[(i,g)])
            model.Add(assignments[i] != g).OnlyEnforceIf(x[(i,g)].Not())
    # Each player is assigned to exactly one group
    for i in range(n):
        model.Add(sum(x[(i,g)] for g in range(num_groups)) == 1)

    # Each group has exactly 4 players
    for g in range(num_groups):
        model.Add(sum(x[(i,g)] for i in range(n)) == 4)
    
    pairings = {(i,j) : 0 for i in range(n) for j in range(n) if i < j}
    for game in game_history:
        idxs = sorted([id_to_idx[pid] for pid in game.get_pids() if pid in id_to_idx])
        for i in range(len(idxs)):
            for j in range(i+1, len(idxs)):
                try:
                    pairings[(idxs[i], idxs[j])] += 1
                except KeyError:
                    print(game)
                    print(i,j)
                    print(idxs)
                    print(id_to_idx)
                    assert False
    
    for (i,j), count in pairings.items():
        assert count <= 2
        if count == 2:
            model.Add(assignments[i] != assignments[j])
    
    matched = {}
    for i in range(n):
        for j in range(i+1, n):
            b = model.NewBoolVar(f"matched_{i}_{j}")
            model.Add(assignments[i] == assignments[j]).OnlyEnforceIf(b)
            model.Add(assignments[i] != assignments[j]).OnlyEnforceIf(b.Not())
            matched[(i,j)] = b
    
    for i in range(n):
        new_partners = []
        for j in range(n):
            if i == j:
                continue
            tuple_ij = (i,j) if i < j else (j,i)
            if pairings[tuple_ij] == 0:
                new_partners.append(matched[tuple_ij])
        model.Add(sum(new_partners) >= 2)
    
    pairwise_rating_prods = {(i,j): players[i].latest_elo * players[j].latest_elo for i in range(n) for j in range(n) if i < j}
    model.Maximize(sum(pairwise_rating_prods[(i,j)] * matched[(i,j)] for i in range(n) for j in range(i+1, n)))
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if not status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('No solution found.')
        assert False
    
    games = [[] for _ in range(num_groups)]
    game_objs = []
    for index, var in enumerate(assignments):
        print(f'{players[index]} = {solver.Value(var)}')
        games[solver.Value(var)].append(players[index])
    for pool in games:
        sorted_pool = sorted(pool, key=lambda p: p.id)
        game_objs.append(Game(round = round_num, p1=sorted_pool[0].id, p2=sorted_pool[1].id, p3=sorted_pool[2].id, p4=sorted_pool[3].id, result=-1))
    return game_objs

In [16]:
generate_matchings(players[:8], [Game(round = 0, p1=10, p2=11, p3=12, p4=13, result=1)], round_num = 1)

[Player(id=10, name='Josh', email='', initial_elo=1500, latest_elo=1775, games_played=0), Player(id=11, name='Kyan', email='', initial_elo=1500, latest_elo=1718, games_played=0), Player(id=12, name='Jake', email='', initial_elo=1500, latest_elo=1641, games_played=0), Player(id=13, name='Jai', email='', initial_elo=1500, latest_elo=1559, games_played=0), Player(id=14, name='Archie', email='', initial_elo=1500, latest_elo=1535, games_played=0), Player(id=15, name='Steven', email='', initial_elo=1500, latest_elo=1565, games_played=0), Player(id=16, name='Aubrey', email='', initial_elo=1500, latest_elo=1439, games_played=0), Player(id=17, name='Reece', email='', initial_elo=1500, latest_elo=1438, games_played=0)]
Player(id=10, name='Josh', email='', initial_elo=1500, latest_elo=1775, games_played=0) = 0
Player(id=11, name='Kyan', email='', initial_elo=1500, latest_elo=1718, games_played=0) = 0
Player(id=12, name='Jake', email='', initial_elo=1500, latest_elo=1641, games_played=0) = 1
Playe

[Game(round=1, p1=10, p2=11, p3=14, p4=15, result=-1),
 Game(round=1, p1=12, p2=13, p3=16, p4=17, result=-1)]

# Managing Tournament

In [82]:
class TourneyManager:
    def __init__(self):
        self.players = []
        self.id_to_player = {}
        self.pool_history = [] # one list per round, each list contains Game objects
        self.is_active = {} # player_id -> bool
        self.current_round = 0
        self.load_players_from_form()
        self.games = []
    
    def refresh_game_history(self):
        self.games = get_game_history()
    
    def refresh_players_list(self):
        resps = get_form_responses()
        assert len(resps["UCLA email"].unique()) == len(resps)
        c_email  = "UCLA email"
        c_name = "First and Last name"
        c_skill = "What is your skill level"
        for index, row in resps.iterrows():
            skill = int(row[c_skill][0])
            initial_elo = 600 + skill * 100
            player = Player(index+2, row[c_name], row[c_email], initial_elo)
            if player.id not in self.is_active:
                self.add_player(player)
    
    def update_ratings(self, refresh_games = True):
        if refresh_games:
            self.refresh_game_history()
        optimize_ratings(self.players, self.games, 1000)
    
    def load_players_from_form(self):
        resps = get_form_responses()
        assert len(resps["UCLA email"].unique()) == len(resps)
        c_email  = "UCLA email"
        c_name = "First and Last name"
        c_skill = "What is your skill level"
        for index, row in resps.iterrows():
            skill = int(row[c_skill][0])
            initial_elo = 600 + skill * 100
            player = Player(index+2, row[c_name], row[c_email], initial_elo)
            self.add_player(player)
        
    def make_new_round(self, manual_assigned_games = []):
        pids = set([pid for game in manual_assigned_games for pid in game.get_pids()])
        assert len(pids) == len(manual_assigned_games) * 4
        for pid in pids:
            assert pid in self.is_active

        self.update_ratings()
        active_players = [i for i in self.get_active_players() if i.id not in pids]
        assert len(active_players) % 4 == 0

        
        recent_games = [game for round in self.pool_history for game in round if game.round >= self.current_round - 1]
        self.current_round += 1

        try:
            print("Trying to get new matchings with recent games:", recent_games)
            new_pools = generate_matchings(active_players, recent_games, self.current_round)
        except:
            print("Failed to get new matchins, loosening constraints")
            try:
                loosened = [game for game in recent_games if game.round >= self.current_round - 1]
                print("Loosened:", loosened)
                new_pools = generate_matchings(active_players, loosened, self.current_round)
            except:
                print("Failed again, removing constraints")
                new_pools = generate_matchings(active_players, [], self.current_round)
        
        self.pool_history.append(new_pools)

        for index, game in enumerate(new_pools):
            post_pools(game, index + 1)

    def add_player(self, player: Player):
        self.players.append(player)
        self.is_active[player.id] = False
        self.id_to_player[player.id] = player
    
    def make_active(self, pid : int):
        assert pid in self.is_active
        if not self.is_active[pid]:
            self.is_active[pid] = True
            print(f"{pid}:{self.id_to_player[pid].name} is now active.")
        else:
            print(f"{pid} is already active.")
    
    def make_inactive(self, pid : int):
        assert pid in self.is_active
        if self.is_active[pid]:
            self.is_active[pid] = False
            print(f"{pid}: {self.id_to_player[pid].name} is now inactive.")
        else:
            print(f"{pid} is already inactive.")
    
    def get_active_players(self):
        return [p for p in self.players if self.is_active[p.id]]
    
    def get_num_active_players(self):
        return len(self.get_active_players())

    def activate_players_session(self):
        print("Activating players session")
        print("To search for an ID by email enter: search <email>")
        print("To activate a player enter: add <id>")
        print("To deactivate a player enter: unadd <id>")
        print("When done activating players enter: done")
        while True:
            cmd = input("Enter command (or 'done' to finish): ").strip().lower().split()
            if cmd == []:
                continue
            if cmd[0] == "search":
                search_str = cmd[1]
                matching_emails = [f"{p.id}: {p.email}" for p in self.players if search_str in p.email]
                print("Matching emails:")
                for me in matching_emails[:min(5, len(matching_emails))]:
                    print(me)
                if len(matching_emails) > 5:
                    print(f"too many matches")
            elif cmd[0] == "add":
                player_id = int(cmd[1])
                self.make_active(player_id)
            elif cmd[0] == "unadd":
                player_id = int(cmd[1])
                self.make_inactive(player_id)
            elif cmd[0] == "done":
                break
            else:
                print("Unknown command. Please try again.")
    
    def test_add_examples(self):
        i = 94
        while i <= 105:
            self.make_active(i)
            i += 1


# Running the Tournament

In [83]:
t = TourneyManager()